In [1]:
# ======================
# 🧩 Imports and Paths
# ======================
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Paths
train_dir = 'train'  
test_dir = 'test'    

# Image size
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

### EfficientNetB0

In [2]:
# ======================
# 🧠 EfficientNetB0 - Frozen
# ======================

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='training')
val_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='validation')


Found 560 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [3]:
# Base model
base_model_b0 = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model_b0.trainable = False  # freeze layers

x = GlobalAveragePooling2D()(base_model_b0.output)
x = Dropout(0.4)(x)
output = Dense(1, activation='sigmoid')(x)
model_b0 = Model(inputs=base_model_b0.input, outputs=output)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [4]:
# Compile
model_b0.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train
history_b0 = model_b0.fit(train_generator, validation_data=val_generator, epochs=10)

c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 30s 522ms/step - accuracy: 0.5067 - loss: 0.7019 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 458ms/step - accuracy: 0.4973 - loss: 0.6918 - val_accuracy: 0.5000 - val_loss: 0.6929
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 431ms/step - accuracy: 0.4878 - loss: 0.7075 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 460ms/step - accuracy: 0.4978 - loss: 0.7008 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 446ms/step - accuracy: 0.5228 - loss: 0.6915 - val_accuracy: 0.5000 - val_loss: 0.6929
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 442ms/step - accuracy: 0.4806 - loss: 0.7035 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 470ms/step - accuracy: 0.4925 - loss: 0.6993 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 429ms/step - accuracy: 0.4457 - loss: 0.7079 - val_accu

In [5]:
from sklearn.metrics import accuracy_score
import numpy as np
# Get validation predictions
val_generator.reset()
val_preds = model_b0.predict(val_generator)
val_preds_classes = (val_preds > 0.5).astype(int)

# True labels
true_labels = val_generator.classes

# Compute accuracy
val_accuracy = accuracy_score(true_labels, val_preds_classes)
print(f"✅ Validation Accuracy: {val_accuracy:.4f}")

9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 687ms/step
✅ Validation Accuracy: 0.5000


In [7]:
# Predict test set
test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=1, class_mode=None, shuffle=False)
preds_b0 = model_b0.predict(test_gen)
results_b0 = pd.DataFrame({
    'image file': [os.path.basename(f) for f in test_gen.filenames],
    'label': ['M' if p > 0.5 else 'N' for p in preds_b0]
})
results_b0.to_csv('EfficientNetB0_predictions.csv', index=False)
print("✅ Saved EfficientNetB0_predictions.csv")

Found 300 images belonging to 1 classes.


c:\Users\pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


300/300 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step
✅ Saved EfficientNetB0_predictions.csv


### EfficientNetB0 fine tuning 

In [8]:
# ======================
# ⚙️ EfficientNetB0 Fine-Tuning + Strong Augmentation
# ======================

# Data augmentation
train_datagen_aug = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)
train_gen_aug = train_datagen_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='training')
val_gen_aug = train_datagen_aug.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary', subset='validation')


Found 560 images belonging to 2 classes.
Found 140 images belonging to 2 classes.


In [9]:
# Base model
base_model_ft = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model_ft.trainable = False

x = GlobalAveragePooling2D()(base_model_ft.output)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
model_ft = Model(inputs=base_model_ft.input, outputs=output)

model_ft.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Stage 1: Train frozen base
history_ft_1 = model_ft.fit(train_gen_aug, validation_data=val_gen_aug, epochs=10)

# Stage 2: Gradual unfreezing (fine-tune last 20 layers)
base_model_ft.trainable = True
for layer in base_model_ft.layers[:-20]:
    layer.trainable = False

model_ft.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_sched = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)

history_ft_2 = model_ft.fit(train_gen_aug, validation_data=val_gen_aug, epochs=20,
                            callbacks=[early_stop, lr_sched])

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 33s 602ms/step - accuracy: 0.5098 - loss: 0.7088 - val_accuracy: 0.5000 - val_loss: 0.6941
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 18s 527ms/step - accuracy: 0.4794 - loss: 0.7000 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 539ms/step - accuracy: 0.5444 - loss: 0.6923 - val_accuracy: 0.4929 - val_loss: 0.6931
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 18s 524ms/step - accuracy: 0.5054 - loss: 0.7036 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 19s 535ms/step - accuracy: 0.4950 - loss: 0.7052 - val_accuracy: 0.5000 - val_loss: 0.6933
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 21s 609ms/step - accuracy: 0.5086 - loss: 0.7034 - val_accuracy: 0.5000 - val_loss: 0.6933
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 29s 827ms/step - accuracy: 0.5105 - loss: 0.7004 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 21s 591ms/step - accuracy: 0.5289 - loss: 0.7006 - val_accu

In [11]:
# Predict test set
preds_ft = model_ft.predict(test_gen)
results_ft = pd.DataFrame({
    'image file': [os.path.basename(f) for f in test_gen.filenames],
    'label': ['M' if p > 0.5 else 'N' for p in preds_ft]
})
results_ft.to_csv('EfficientNetB0_Finetuned_predictions.csv', index=False)
print("✅ Saved EfficientNetB0_Finetuned_predictions.csv")

300/300 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step
✅ Saved EfficientNetB0_Finetuned_predictions.csv
